In [1]:
import numpy as np

load_path = '../global_data/time_9830400(76800x128)x9x9/'
save_path = './data/'

In [2]:
# trials = np.load(load_path + 'trials.npy')
# bases = np.load(load_path + 'bases.npy')
# labels = np.load(load_path + 'labels.npy')
# print(trials.shape, bases.shape, labels.shape)

In [3]:
window_size = 128

In [4]:
# # 切窗口
# def slice(trials):
#     samples = []
#     for trial in trials:
#         start = 0
#         while start + window_size <= trial.shape[0]:
#             samples.append(trial[start : start + window_size])
#             start = start + window_size
#     return np.array(samples)

In [5]:
# trial_samples = slice(trials)
# bases_samples = slice(bases)


# np.save(save_path + 'trials.npy', trial_samples)
# np.save(save_path + 'bases.npy', bases_samples)
# print(trial_samples.shape, bases_samples.shape)

In [6]:
trial_samples = np.load(save_path + 'trials.npy')
bases_samples = np.load(save_path + 'bases.npy')
labels = np.load(load_path + 'labels.npy')
print(trial_samples.shape, bases_samples.shape, labels.shape)

(76800, 128, 9, 9) (3840, 128, 9, 9) (1280, 2)


In [7]:
# 去基线
def de_base():
    global bases_samples, trial_samples
    bases_samples = bases_samples.reshape((1280, 3, 128, 9, 9))
    bases_samples = bases_samples.mean(axis = 1)

    for i, base in enumerate(bases_samples):
        trial_samples[i * 60 : (i + 1) * 60] = trial_samples[i * 60 : (i + 1) * 60] - base

de_base()

In [8]:
# 离散化标签
labels = np.where(labels >= 5, 1, 0)

# 复制以对齐样本
labels = np.repeat(labels, 60, axis = 0)
print(labels.shape)

(76800, 2)


In [9]:
# 数据集划分
def data_split(features, labels, train_ratio = 0.9):
    print('split dataset...')
    sample_cnt = labels.shape[0]
    shuffer_list = np.arange(sample_cnt)
    np.random.shuffle(shuffer_list)

    features = features[shuffer_list] # trials_de_base
    labels = labels[shuffer_list]

    x_train = features[:int(train_ratio * sample_cnt)]
    y_train = labels[:int(train_ratio * sample_cnt)]
    x_test = features[int(train_ratio * sample_cnt):]
    y_test = labels[int(train_ratio * sample_cnt):]
    
    return x_train, y_train, x_test, y_test

x_train, y_train, x_vali, y_vali = data_split(trial_samples, labels, train_ratio = 0.9)

split dataset...


In [10]:
def z_norm(x_train, x_test, window_size):# z归一化
    print('z_norm...')
    chan_to_1020={0:[0,3],1:[1,3],2:[2,2],3:[2,0],4:[3,1],5:[3,3],6:[4,2],7:[4,0],8:[5,1],
                  9:[5,3],10:[6,2],11:[6,0],12:[7,3],13:[8,3],14:[8,4],15:[6,4],16:[0,5],
                  17:[1,5],18:[2,4],19:[2,6],20:[2,8],21:[3,7],22:[3,5],23:[4,4],24:[4,6],
                    25:[4,8],26:[5,7],27:[5,5],28:[6,6],29:[6,8],30:[7,5],31:[8,5]}

    # 需要归一化的位置
    need_norm_pos = np.zeros(shape=(9,9), dtype=int)

    for val in chan_to_1020.values():
        need_norm_pos[val[0]][val[1]] = 1
    # print(need_norm_pos)

    for i in range(9):
        for j in range(9):
            if need_norm_pos[i][j] == 0:
                continue
            for t in range(window_size):
                mean = np.mean(x_train[:, t, i, j])
                std = np.std(x_train[:, t, i, j])
                x_train[:, t, i, j] -= mean
                x_train[:, t, i, j] /= std
                x_test[:, t, i, j] -= mean
                x_test[:, t, i, j] /= std
    return x_train, x_test

x_train, x_vali = z_norm(x_train, x_vali, window_size)

z_norm...


In [11]:
emotion_dim = 0
y_train =y_train[:, emotion_dim]
y_vali = y_vali[:, emotion_dim]

In [12]:
from torch.utils.data import TensorDataset
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import nn
# from torchviz import make_dot
from torch.autograd import Variable

In [13]:
# 超参设置
batch_size = 128
max_epoch = 1000

lr = 0.00001
patient = 10

In [14]:
print('make tensor...')
x_train = torch.tensor(x_train, dtype=torch.float).unsqueeze(1) # 做成3d卷积的输入形状 [batch_size, channel, depth, height, width]
y_train = torch.tensor(y_train, dtype=torch.long)
x_vali = torch.tensor(x_vali, dtype=torch.float).unsqueeze(1)
y_vali = torch.tensor(y_vali, dtype=torch.long)

data_train = TensorDataset(x_train, y_train)
train_loader = DataLoader(dataset=data_train, batch_size=batch_size, shuffle=True)

make tensor...


In [15]:
device = torch.device('cuda')       # 用GPU来运行

In [16]:
class Conv3d(nn.Module):
    def __init__(self):
        super(Conv3d, self).__init__()
        self.conv1 = nn.Conv3d(in_channels = 1, out_channels = 1, kernel_size = (3, 3, 3),
            stride = (1, 1, 1), padding = (0, 0, 0), device=device)
        
        self.conv2 = nn.Conv3d(in_channels = 1, out_channels = 1, kernel_size = (3, 3, 3),
            stride = (1, 1, 1), padding = (0, 0, 0), device=device)
        
        self.fc1 = nn.Linear(3100, 2560)
        self.fc2 = nn.Linear(2560, 128)
        self.fc3 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [17]:
class  Inception_Conv3d(nn.Module):
    def __init__(self):
        super(Inception_Conv3d, self).__init__()
        self.conv11 = nn.Conv3d(in_channels = 1, out_channels = 1, kernel_size = (3, 3, 3),
            stride = (1, 1, 1), padding = (0, 0, 0))
        
        self.conv12 = nn.Conv3d(in_channels = 1, out_channels = 1, kernel_size = (9, 3, 3),
            stride = (1, 1, 1), padding = (0, 0, 0))
        
        self.conv13 = nn.Conv3d(in_channels = 1, out_channels = 1, kernel_size = (27, 3, 3),
            stride = (1, 1, 1), padding = (0, 0, 0))
        
        
        self.conv21 = nn.Conv3d(in_channels = 3, out_channels = 1, kernel_size = (3, 3, 3),
            stride = (1, 1, 1), padding = (0, 0, 0))
        
        self.conv22 = nn.Conv3d(in_channels = 3, out_channels = 1, kernel_size = (9, 3, 3),
            stride = (1, 1, 1), padding = (0, 0, 0))
        
        self.conv23 = nn.Conv3d(in_channels = 3, out_channels = 1, kernel_size = (27, 3, 3),
            stride = (1, 1, 1), padding = (0, 0, 0))
        
        
        self.fc1 = nn.Linear(8550, 2560)
        self.fc2 = nn.Linear(2560, 128)
        self.fc3 = nn.Linear(128, 2)
        
    def forward(self, x):
        x1 = F.relu(self.conv11(x))
        x2 = F.relu(self.conv12(x))
        x3 = F.relu(self.conv13(x))
        
        x2 = F.pad(x2, (0, 0, 0, 0, 0, x1.shape[2] - x2.shape[2]), "constant", 0)
        x3 = F.pad(x3, (0, 0, 0, 0, 0, x1.shape[2] - x3.shape[2]), "constant", 0)
        
        x = torch.concat([x1, x2, x3], dim = 1)
        
        
        x1 = F.relu(self.conv21(x))
        x2 = F.relu(self.conv22(x))
        x3 = F.relu(self.conv23(x))
        
#         x2 = F.pad(x2, (0, 0, 0, 0, 0, x1.shape[2] - x2.shape[2]), "constant", 0)
#         x3 = F.pad(x3, (0, 0, 0, 0, 0, x1.shape[2] - x3.shape[2]), "constant", 0)

        x1 = x1.view(x.shape[0], -1)
        x2 = x2.view(x.shape[0], -1)
        x3 = x3.view(x.shape[0], -1)
        
        x = torch.concat([x1, x2, x3], dim = 1)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

- 时空多尺度感知，时间多尺度感知，空间也多尺度感知，改变卷积核正方形的变长。
- 做padding
- 加batch normal
- 加Dropout
- 调整batch_size、学习率
- 调整参数量
- 使用1x1的卷积核，增加网络深度
- 设计一个损失函数

# 对比实验

- 使用不同的window_size
- 能不能设计一个损失函数，比较你的损失和交叉熵的性能
- 多数据集验证
- 消融实验

# 实验结果展示
- 时间、内存分析
- 混淆矩阵
- 多评估指标比较
- 模型中间过程验证模型设计
- 与其他方法的比较

# 来这里初始化一下模型参数

In [18]:
# 初始化权重参数

# 1. 根据网络层的不同定义不同的初始化方式     
def weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0)
    # 也可以判断是否为conv2d，使用相应的初始化方式 
    elif isinstance(m, nn.Conv3d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
     # 是否为批归一化层
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)

model = Inception_Conv3d().to(device)

# 3. 将weight_init应用在子模块上
model.apply(weight_init)

Inception_Conv3d(
  (conv11): Conv3d(1, 1, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv12): Conv3d(1, 1, kernel_size=(9, 3, 3), stride=(1, 1, 1))
  (conv13): Conv3d(1, 1, kernel_size=(27, 3, 3), stride=(1, 1, 1))
  (conv21): Conv3d(3, 1, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv22): Conv3d(3, 1, kernel_size=(9, 3, 3), stride=(1, 1, 1))
  (conv23): Conv3d(3, 1, kernel_size=(27, 3, 3), stride=(1, 1, 1))
  (fc1): Linear(in_features=8550, out_features=2560, bias=True)
  (fc2): Linear(in_features=2560, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)

In [19]:
opt = torch.optim.Adam(model.parameters(), lr)
criterion = nn.CrossEntropyLoss().to(device)

In [20]:
model_save_path = 'model/'

# 提前终止
acc_history=0
step=0

for i in range(max_epoch):
    model.train()
    print(f'epoch:{i}', end = ', ')
    epoch_loss = 0
    epoch_acc = 0
    for batch_idx, batch_data in enumerate(train_loader):
        x, y = batch_data
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        epoch_loss = epoch_loss + loss
        out = torch.max(out, 1)[1]
        epoch_acc = epoch_acc + (out == y).sum()
        
#         if batch_idx % 1000 == 0:
#             train_loss = loss / x.shape[0]
#             out = torch.max(out, 1)[1]
#             train_acc = (out == y).sum() / x.shape[0]
#             print(f'batch: {batch_idx}, train_loss: {train_loss:.6f}, train_acc: {train_acc:.6f}')
    print(f'train_loss:{epoch_loss / len(train_loader.dataset) :.6f}, train_acc:{epoch_acc / len(train_loader.dataset) :.6f}', end = ', ')
    
    # 测试
    model.eval()
    x_vali, y_vali = x_vali.to(device), y_vali.to(device)
    out = model(x_vali)
    vali_loss = criterion(out, y_vali) / x_vali.shape[0]

    out = torch.max(out, 1)[1]
    vali_acc = (out == y_vali).sum() / x_vali.shape[0]

    print(f'vali_loss: {vali_loss:.6f}, vali_acc: {vali_acc:.6f}')
    
#     out = model(x)
#     vali_loss = criterion(out, y) / y.shape[0]

#     out = torch.max(out, 1)[1]
#     vali_acc = (out == y).sum() / x.shape[0]

#     print(f'vali_loss: {vali_loss:.6f}, vali_acc: {vali_acc:.6f}')

    step = step + 1
    #模型性能有所提升则保存模型，并更新loss_value
    if  vali_acc > acc_history:
        step = 0
        torch.save(model.state_dict(), model_save_path + f'model_parameter.pkl')
        acc_history = vali_acc

    if step >= patient:
        break

epoch:0, train_loss:0.004703, train_acc:0.672584, vali_loss: 0.000069, vali_acc: 0.745703
epoch:1, train_loss:0.003560, train_acc:0.795255, vali_loss: 0.000061, vali_acc: 0.807162
epoch:2, train_loss:0.002800, train_acc:0.859404, vali_loss: 0.000052, vali_acc: 0.831901
epoch:3, train_loss:0.002255, train_acc:0.894835, vali_loss: 0.000046, vali_acc: 0.861719
epoch:4, train_loss:0.001824, train_acc:0.922772, vali_loss: 0.000048, vali_acc: 0.842448
epoch:5, train_loss:0.001489, train_acc:0.942679, vali_loss: 0.000045, vali_acc: 0.856510
epoch:6, train_loss:0.001219, train_acc:0.957407, vali_loss: 0.000040, vali_acc: 0.884245
epoch:7, train_loss:0.000992, train_acc:0.968953, vali_loss: 0.000039, vali_acc: 0.890104
epoch:8, train_loss:0.000808, train_acc:0.977575, vali_loss: 0.000036, vali_acc: 0.901302
epoch:9, train_loss:0.000650, train_acc:0.984896, vali_loss: 0.000034, vali_acc: 0.908073
epoch:10, train_loss:0.000552, train_acc:0.987905, vali_loss: 0.000037, vali_acc: 0.896354
epoch:11,

In [21]:
# 经过测试，不去基线，训练集收敛很慢，且训练集准确率可以达到100%，测试集准确率只能达到58%
# 去基线，训练集很快收敛，测试集准确率在第10个epoch就达到了79%

In [22]:
# # 测试

# print('make tensor...')
# x_test = torch.tensor(x_test, dtype=torch.float)
# emo_test = torch.tensor(emo_test, dtype=torch.long)

# model.load_state_dict(torch.load(model_save_path + f'sub_{0}_model_parameter.pkl'))
# model.eval()
# emo_out, id_out = model(x_test, x_test)
# test_emo_loss = Emo_criterion(emo_out, emo_test) / x_test.shape[0]

# emo_out = torch.max(emo_out, 1)[1]
# test_emo_acc = (emo_out == emo_test).sum() / x_test.shape[0]
# #------------------------------------------------看看测试者的id


# print(f'sub: {sub_idx}, test_emo_loss: {vali_emo_loss:.6f}, test_emo_acc: {test_emo_acc:.6f}')